# 第七章: SEC知识图谱对话

- 在本章中，我们将对知识图谱进行更多的探索。
- 首先，通过一些 Cypher 查询直接探索图，然后使用 LangChain 创建一个问答聊天。最后，我们将使用LLM结合这两种技术，以一种有趣的新方式进行操作。

像往常一样，我们将从导入一些库开始，并定义一些将在查询中使用的全局变量。

# 一、环境配置

本教程使用 OpenAI 所开放的 ChatGPT API，因此您需要首先拥有一个 ChatGPT 的 API_KEY（也可以直接访问官方网址在线测试），然后需要安装 OpenAI 的第三方库。为了兼顾简便与兼容性，本教程将介绍在 ```Python 3``` 环境中基于 ```openai.api_key``` 方法的配置。另有基于环境变量的配置方法，详情请参考 [OpenAI 官方文档](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety)。

首先需要安装 OpenAI，LangChain等工具库：
```bash
pip install openai langchain langchain_community langchain_openai
```

## 1.1 导入第三方库

In [ ]:
import os
import textwrap

# OpenAI
import openai

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI

# Warning control
import warnings
warnings.filterwarnings("ignore")

## 1.2 设置环境变量

In [ ]:
# 设置 API_KEY, 请替换成您自己的 API_KEY
openai.api_key = "sk-..."

# 设置 Neo4j, 请替换成您自己的 Neo4j_XXX
NEO4J_URI = ""
NEO4J_USERNAME = ""
NEO4J_PASSWORD = ""
NEO4J_DATABASE = 'neo4j'

# Global constants
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

In [3]:
# 创建知识图谱
kg = Neo4jGraph(
    url=NEO4J_URI, 
    username=NEO4J_USERNAME, 
    password=NEO4J_PASSWORD, 
    database=NEO4J_DATABASE
)

# 二、探索更新后的 SEC 文档图谱
- 下面的一些输出可能与视频略有不同

## 2.1 检查图谱模式

In [4]:
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties are the following: Chunk {textEmbedding:
LIST, f10kItem: STRING, chunkSeqId: INTEGER, text: STRING,
cik: STRING, cusip6: STRING, names: LIST, formId: STRING,
source: STRING, chunkId: STRING},Form {cusip6: STRING,
names: LIST, formId: STRING, source: STRING},Company
{location: POINT, cusip: STRING, names: LIST,
companyAddress: STRING, companyName: STRING, cusip6:
STRING},Manager {location: POINT, managerName: STRING,
managerCik: STRING, managerAddress: STRING},Address
{location: POINT, country: STRING, city: STRING, state:
STRING} Relationship properties are the following: SECTION
{f10kItem: STRING},OWNS_STOCK_IN {shares: INTEGER,
reportCalendarOrQuarter: STRING, value: FLOAT} The
relationships are the following: (:Chunk)-[:NEXT]-
>(:Chunk),(:Chunk)-[:PART_OF]->(:Form),(:Form)-[:SECTION]-
>(:Chunk),(:Company)-[:FILED]->(:Form),(:Company)-
[:LOCATED_AT]->(:Address),(:Manager)-[:LOCATED_AT]-
>(:Address),(:Manager)-[:OWNS_STOCK_IN]->(:Company)


## 2.2 查询经理和地址

- 我们首先随机找到一个经理，然后返回经理和地址，并将其限制为一个。
- 我们找到了 LAKEWOOD，她在纽约市的 Capital Management。
- 注意这里我们不仅向地址节点添加了城市和州，还添加了一个称为位置的东西，其中存储了一个点。这只是存储为数据值的经纬度。这使我们可以进行地理空间搜索和接近搜索。
- 注意：以下查询返回的公司可能与视频中的公司不同

In [5]:
kg.query("""
MATCH (mgr:Manager)-[:LOCATED_AT]->(addr:Address)
RETURN mgr, addr
LIMIT 1
""")

[{'mgr': {'managerCik': '1424381',
   'managerAddress': '650 Madison Avenue, 25th Floor, New York, NY, 10022',
   'location': POINT(-73.9713457 40.7639879),
   'managerName': 'LAKEWOOD CAPITAL MANAGEMENT, LP'},
  'addr': {'country': 'United States',
   'city': 'New York',
   'location': POINT(-73.9821429 40.7584882),
   'state': 'New York'}}]

- 这只是一个随机的经理。如果我们想在数据集中找到特定的人物，比如"royal bank"，我们可以进行文本搜索并返回那个经理。
- 很好，我们找到了经理名称，实际上是加拿大皇家银行。
- 注意我们还得到了一个分数，这是进行全文搜索时的得分，其值范围与向量搜索不同，但原理相同。较高的分数表示更好的匹配。

In [6]:
kg.query("""
  CALL db.index.fulltext.queryNodes(
         "fullTextManagerNames", 
         "royal bank") YIELD node, score
  RETURN node.managerName, score LIMIT 1
""")

[{'node.managerName': 'Royal Bank of Canada', 'score': 4.431276321411133}]

- 我们可以将这两个查询结合起来，首先找到皇家银行，然后找到他们的位置。
- 首先，使用全文搜索找到银行，然后从该经理所在的位置找到地址并返回这些值。毫无意外，加拿大皇家银行在加拿大。

In [7]:
kg.query("""
CALL db.index.fulltext.queryNodes(
         "fullTextManagerNames", 
         "royal bank"
  ) YIELD node, score
WITH node as mgr LIMIT 1
MATCH (mgr:Manager)-[:LOCATED_AT]->(addr:Address)
RETURN mgr.managerName, addr
""")

[{'mgr.managerName': 'Royal Bank of Canada',
  'addr': {'country': 'Canada',
   'city': 'Toronto',
   'location': POINT(-79.3805647 43.6508267),
   'state': 'Ontario'}}]

## 2.3 哪个州拥有最多的投资公司

- 我们可以继续进行更多的探索，看看我们知道哪些信息。例如，我们可以问哪个州拥有最多的投资公司。
- 我们首先匹配一个位于某个地址的经理，然后返回包括该州名称的结果，并通过聚合进行计数，计算出该州出现的次数，称之为经理数量，并将其返回，限制为前10名经理所在的州。

In [8]:
kg.query("""
  MATCH p=(:Manager)-[:LOCATED_AT]->(address:Address)
  RETURN address.state as state, count(address.state) as numManagers
    ORDER BY numManagers DESC
    LIMIT 10
""")

[{'state': 'New York', 'numManagers': 304},
 {'state': 'California', 'numManagers': 302},
 {'state': 'Massachusetts', 'numManagers': 146},
 {'state': 'Pennsylvania', 'numManagers': 138},
 {'state': 'Texas', 'numManagers': 125},
 {'state': 'Illinois', 'numManagers': 121},
 {'state': 'Florida', 'numManagers': 115},
 {'state': 'Connecticut', 'numManagers': 77},
 {'state': 'Ohio', 'numManagers': 76},
 {'state': 'New Jersey', 'numManagers': 69}]

- 我们看到很多人在纽约，很多管理公司在加利福尼亚州，还有一些在其他州。

## 2.4 哪个州拥有最多的公司

- 我们可以问同样的问题，但这次是关于上市公司。
- 我们首先写出从公司到地址的模式匹配，列出地址所在的州，然后对地址州进行聚合，并将其作为公司的数量。

In [9]:
kg.query("""
  MATCH p=(:Company)-[:LOCATED_AT]->(address:Address)
  RETURN address.state as state, count(address.state) as numCompanies
    ORDER BY numCompanies DESC
""")

[{'state': 'California', 'numCompanies': 7},
 {'state': 'Delaware', 'numCompanies': 1},
 {'state': 'New York', 'numCompanies': 1},
 {'state': 'Oregon', 'numCompanies': 1}]

- 好的，看起来在管理投资公司和公司之间，至少在这个样本数据集中，大多数都在加利福尼亚州。

## 2.5 加州哪些城市拥有最多的投资公司

- 让我们进一步深入加利福尼亚。我们可能会问到：加利福尼亚哪些城市拥有最多的投资公司？
- 我们可以继续采用相同的方法，从位于某个地址的经理的模式匹配开始，然后我们将地址限制在加利福尼亚州。按照相同的想法，返回城市名称，计算城市出现的次数，并返回前10个。

In [10]:
kg.query("""
  MATCH p=(:Manager)-[:LOCATED_AT]->(address:Address)
         WHERE address.state = 'California'
  RETURN address.city as city, count(address.city) as numManagers
    ORDER BY numManagers DESC
    LIMIT 10
""")

[{'city': 'San Francisco', 'numManagers': 48},
 {'city': 'Los Angeles', 'numManagers': 44},
 {'city': 'San Diego', 'numManagers': 17},
 {'city': 'Pasadena', 'numManagers': 13},
 {'city': 'Menlo Park', 'numManagers': 9},
 {'city': 'Newport Beach', 'numManagers': 9},
 {'city': 'Irvine', 'numManagers': 9},
 {'city': 'Walnut Creek', 'numManagers': 8},
 {'city': 'Palo Alto', 'numManagers': 6},
 {'city': 'Lafayette', 'numManagers': 6}]

- 我们可以看到，这里有一些北加州和南加州的竞争。北部和南部都很密集，其他地方则混合分布。

## 2.6 加州哪个城市拥有最多的上市公司

- 现在我们对公司问同样的问题，加利福尼亚的哪些城市有最多的公司？

In [11]:
kg.query("""
  MATCH p=(:Company)-[:LOCATED_AT]->(address:Address)
         WHERE address.state = 'California'
  RETURN address.city as city, count(address.city) as numCompanies
    ORDER BY numCompanies DESC
""")

[{'city': 'Santa Clara', 'numCompanies': 3},
 {'city': 'San Jose', 'numCompanies': 2},
 {'city': 'Sunnyvale', 'numCompanies': 1},
 {'city': 'Cupertino', 'numCompanies': 1}]

- 这次是一个非常不同的结果。
- 我们知道这个样本数据集中只有10家公司，它们在圣克拉拉、圣何塞、桑尼维尔和库比蒂诺。看起来上市公司和经理在完全不同的城市。

## 2.7 旧金山的顶级投资公司有哪些

- 我们注意有很多投资公司在旧金山。我们现在可以深入探讨旧金山，看看在旧金山有哪些顶级管理公司。
- 这与之前的模式相同，对于位于某个地址的经理。所以我们会说城市必须是旧金山。
- 除了经理名称外，还有该公司投资的内容，将所有这些关系的值属性相加，称之为总投资价值。然后返回，限制为前10名，按降序排列。

In [12]:
kg.query("""
  MATCH p=(mgr:Manager)-[:LOCATED_AT]->(address:Address),
         (mgr)-[owns:OWNS_STOCK_IN]->(:Company)
         WHERE address.city = "San Francisco"
  RETURN mgr.managerName, sum(owns.value) as totalInvestmentValue
    ORDER BY totalInvestmentValue DESC
    LIMIT 10
""")

[{'mgr.managerName': 'Dodge & Cox', 'totalInvestmentValue': 3889236092000.0},
 {'mgr.managerName': 'WELLS FARGO & COMPANY/MN',
  'totalInvestmentValue': 2177580039000.0},
 {'mgr.managerName': 'CHARLES SCHWAB INVESTMENT MANAGEMENT INC',
  'totalInvestmentValue': 1944847519000.0},
 {'mgr.managerName': 'Parallax Volatility Advisers, L.P.',
  'totalInvestmentValue': 694023723000.0},
 {'mgr.managerName': 'PARNASSUS INVESTMENTS, LLC',
  'totalInvestmentValue': 211068925000.0},
 {'mgr.managerName': 'Spyglass Capital Management LLC',
  'totalInvestmentValue': 98135259000.0},
 {'mgr.managerName': 'Valiant Capital Management, L.P.',
  'totalInvestmentValue': 52124040000.0},
 {'mgr.managerName': 'Ensemble Capital Management, LLC',
  'totalInvestmentValue': 42355370000.0},
 {'mgr.managerName': 'Woodline Partners LP',
  'totalInvestmentValue': 41497384000.0},
 {'mgr.managerName': 'Alta Park Capital, LP',
  'totalInvestmentValue': 38959909000.0}]

- 这是旧金山的顶级投资公司名单，我们可能会认出其中一些名字。

## 2.8 圣克拉拉有哪些公司

- 回到公司名单，在这个样本集中，大多数在圣克拉拉，让我们找出这些公司是谁。
- 这很简单。我们将从公司开始，它们位于某个地址，地址城市是圣克拉拉。

In [13]:
kg.query("""
  MATCH (com:Company)-[:LOCATED_AT]->(address:Address)
         WHERE address.city = "Santa Clara"
  RETURN com.companyName
""")

[{'com.companyName': 'PALO ALTO NETWORKS INC'},
 {'com.companyName': 'SEAGATE TECHNOLOGY'},
 {'com.companyName': 'ATLASSIAN CORP PLC'}]

## 2.9 圣克拉拉附近有什么公司

- 到目前为止，我们一直在使用显式关系探索图谱。
- 我们还可以基于它们的位置坐标查找内容。记住，我们添加了一个地理空间索引。这变得非常有趣。这有点像在二维空间中进行向量搜索，但是使用笛卡尔距离而不是余弦相似度。
- 我们可以问哪些公司在圣克拉拉附近，这类似于哪些公司在圣克拉拉，但我们不希望它们在圣克拉拉，而是在附近。
- 我们如何处理这个问题呢？首先，我们匹配一个地址，称为 sc，我们希望 sc 的城市是圣克拉拉，然后我们希望公司位于某个公司地址。
- WHERE子句说明使用point.distance，这是一个内置在Cypher中的距离函数，point.distance从两个不同的位置，sc位置（即圣克拉拉的位置），然后是任何公司地址的位置（这家公司的位置）。
- 我们希望它少于10,000，这是10,000米。所有的距离都以米为单位。符合条件的，我们将返回公司名称和公司节点中列出的完整地址。

In [14]:
kg.query("""
  MATCH (sc:Address)
    WHERE sc.city = "Santa Clara"
  MATCH (com:Company)-[:LOCATED_AT]->(comAddr:Address)
    WHERE point.distance(sc.location, comAddr.location) < 10000
  RETURN com.companyName, com.companyAddress
""")

[{'com.companyName': 'PALO ALTO NETWORKS INC',
  'com.companyAddress': '3000 Tannery Way, Santa Clara, CA 95054, USA'},
 {'com.companyName': 'GSI TECHNOLOGY INC',
  'com.companyAddress': '1213 Elko Dr, Sunnyvale, CA 94089, USA'},
 {'com.companyName': 'SEAGATE TECHNOLOGY',
  'com.companyAddress': '2445 Augustine Dr, Santa Clara, CA 95054, USA'},
 {'com.companyName': 'ATLASSIAN CORP PLC',
  'com.companyAddress': '431 El Camino Real, Santa Clara, CA 95050, USA'},
 {'com.companyName': 'APPLE INC', 'com.companyAddress': 'Cupertino, CA, USA'}]

- 我们可以看到Palo Alto Networks位于圣克拉拉，当然在圣克拉拉附近。
- 我们还可以看到桑尼维尔，圣克拉拉再次出现，然后库比蒂诺的苹果公司出现了。
- 所以现在我们有两种不同的方式。要么我们确切知道我们想要什么，比如圣克拉拉附近的人，或者离圣克拉拉近的东西，那就是一个距离函数。
- 回想一下更新的模式，我们有这些新节点标记为地址。这些地址节点只有城市、州和国家。公司和管理公司通过一个位于关系连接到这些地址。

## 2.10 圣克拉拉附近有哪些投资公司

- 当然，我们可以对管理公司提出相同的问题。我们将这些结果留在这里，尝试相同的查询。
- 我们将匹配地址，然后对于地址是圣克拉拉，匹配一个位于某个地址的经理，然后进行point.distance计算。同样限制在10,000米以内。- 看来附近只有一家管理公司，他们在加利福尼亚州坎贝尔。

In [15]:
kg.query("""
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress
""")

[{'mgr.managerName': 'Mine & Arao Wealth Creation & Management, LLC.',
  'mgr.managerAddress': '901 CAMPISI WAY, SUITE 140, CAMPBELL, CA, 95008'}]

- 我们可以尝试扩大范围，看看我们能找到什么？
- 结果和预期一样，范围越大，我们找到的管理公司越多。

## 2.11 Palo Alto Networks 附近有哪些投资公司

- 我们可以像之前一样，将这些单独的查询结合起来，做更有趣的事情。例如，我们可以问，不是某个位置附近的投资公司，而是某个我们知道的公司附近的投资公司。我们可能记得看到Palo Alto Networks。
- 即使我拼错了名字，也没关系。我们会运行一个查询，即使有拼写错误，找到公司，然后找到该公司附近的管理公司。
- 我们从全文查询开始。我们将通过公司名称进行全文搜索，查找拼写错误的Palo Alto networks。
- 我们将获取该查询的节点及其评分。这里我们将其重命名为com，因为我们知道它代表一个公司。然后进行模式匹配，从公司位于某个公司地址。
- 类似地，找到位于某个经理地址的经理。注意这两个模式没有任何连接。我们有一个WHERE子句说找到point.distance在公司地址位置和经理地址位置之间。我们将其限制在10K范围内，然后返回经理并将距离转换为公里。

In [16]:
# Which investment firms are near Palo Aalto Networks?
kg.query("""
  CALL db.index.fulltext.queryNodes(
         "fullTextCompanyNames", 
         "Palo Aalto Networks"
         ) YIELD node, score
  WITH node as com
  MATCH (com)-[:LOCATED_AT]->(comAddress:Address),
    (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE point.distance(comAddress.location, 
        mgrAddress.location) < 10000
  RETURN mgr, 
    toInteger(point.distance(comAddress.location, 
        mgrAddress.location) / 1000) as distanceKm
    ORDER BY distanceKm ASC
    LIMIT 10
""")

[{'mgr': {'managerCik': '1802994',
   'managerAddress': '901 CAMPISI WAY, SUITE 140, CAMPBELL, CA, 95008',
   'location': POINT(-121.9342655 37.2909459),
   'managerName': 'Mine & Arao Wealth Creation & Management, LLC.'},
  'distanceKm': 6}]

- 即使有拼写错误，我们找到了Palo Alto Networks，找到了管理公司，他们叫Mine and Arrow Wealth Creation and Management, LLC。

- 我们在这节课中读了很多Cypher。这可能会让人一时难以接受。一个好的尝试是暂停视频，并对笔记本中的查询进行一些小的更改。
- 尝试不同的距离、公司名称和城市，看看我们会得到什么。
- 有很多在线资源可以帮助我们学习Cypher，您可以在 neo4j 网站了解有关 Cypher 的更多信息：https://neo4j.com/product/cypher-graph-query-language/。

# 三、用大模型编写 Cypher

- 事实证明，OpenAI的GPT-3.5模型在编写Cypher方面表现相当不错。接下来让我们尝试一下。
- 要让LLM编写 Cypher，我们可以使用一种称为少量示例学习的技术。通过少量示例学习，我们在提示中提供示例，告诉LLM如何完成特定任务。然后我们要求LLM执行该任务。
- 例如，让我们看看一个用于教LLM关于知识图谱和编写 Cypher 的提示。
- 它非常简单，任务是生成Cypher语句以查询图数据库。然后说明只使用模式中的提供的关系类型和属性，不要使用任何其他未提供的关系类型或属性。基本上，我们在请求LLM，请只按照我们的指示，不要偏离。
- 接下来，我们提供实际的模式：在花括号中，知识图谱的模式将被传递到LLM的提示中。所有这些将被打包在一起作为创建提示的标准做法，包含大量指导信息给LLM。
- 请不要包括解释或道歉。只编写Cypher。不要响应与编写Cypher语句无关的问题。不要创建我们没有要求我们做的其他任何内容。
- 最后，我们提供一些示例，这里我们只提供一个为特定问题生成的Cypher示例。我们有一个哈希符号，然后是自然语言中的问题，我们在编写Cypher时见过这个。旧金山有哪些投资公司？然后这是应该编写的Cypher模式，这是位于某个地址的经理，带有一个WHERE子句，这里是一个字符串文字旧金山，而不是传递查询变量，然后只返回经理名称。
- 最后，我们关闭提示，其中包含实际执行的任务。这里是我们的问题。

In [17]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to 
query a graph database.
Instructions:
Use only the provided relationship types and properties in the 
schema. Do not use any other relationship types or properties that 
are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than 
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher 
statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName
The question is:
{question}"""

In [ ]:
CYPHER_GENERATION_TEMPLATE_zh = """任务：生成查询图数据库的 Cypher 语句。
指令：仅使用 Schema 提供的关系类型和属性。请勿使用任何没有提供的其他关系类型或属性。
Schema：{schema} 
注意：请勿在回复中包含任何解释或道歉。除了构建 Cypher 声明之外，请勿回答任何可能提出其他任何问题的问题。
除生成的 Cypher 语句外，请勿包含任何文本。
示例：以下是针对特定问题生成的 Cypher 语句的一些示例：

# 旧金山有哪些投资公司？
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName
问题是：
{question}"""

- 如果有人提出一个问题，比如，圣克拉拉有哪些投资公司？或者其他任何城市，LLM应该从这个示例中学习模式是什么样的，生成的Cypher应该是什么样的，并根据所问的问题，将旧金山替换为圣克拉拉。

为了围绕这一点创建一个工作流程，我们将使用LangChains集成，这非常方便。
- 首先，我们将Cypher生成模板转换为一个Cypher生成提示，使用这个提示模板类。

In [18]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

- 接下来，我们将创建一种新的链。我们之前有问答链，但这是一个不同的链。这是一个GraphCypherQAChain。
- 我们将使用ChatOpenAI作为LLM，图谱是我们用于直接查询 Neo4j 的知识图谱。我们希望它是详细的，告诉我们在执行过程中发生了什么。
- Cypher提示使用的是我们上面创建的Cypher生成提示。

In [19]:
cypherChain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

- 然后，我会添加一个小工具类，以保持整洁。这类似于我们之前所做的，只是使用TextWrap使事情整齐有序。

In [20]:
def prettyCypherChain(question: str) -> str:
    response = cypherChain.run(question)
    print(textwrap.fill(response, 60))

- 我们有一个关于旧金山有哪些投资公司的问题。

In [21]:
prettyCypherChain("What investment firms are in San Francisco?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName
Full Context:
[{'mgr.managerName': 'PARNASSUS INVESTMENTS, LLC'}, {'mgr.managerName': 'SKBA CAPITAL MANAGEMENT LLC'}, {'mgr.managerName': 'ROSENBLUM SILVERMAN SUTTON S F INC /CA'}, {'mgr.managerName': 'CHARLES SCHWAB INVESTMENT MANAGEMENT INC'}, {'mgr.managerName': 'WELLS FARGO & COMPANY/MN'}, {'mgr.managerName': 'Dodge & Cox'}, {'mgr.managerName': 'Strait & Sound Wealth Management LLC'}, {'mgr.managerName': 'Sonoma Private Wealth LLC'}, {'mgr.managerName': 'Fund Management at Engine No. 1 LLC'}, {'mgr.managerName': 'SELDON CAPITAL LP'}]

> Finished chain.
PARNASSUS INVESTMENTS, LLC, ROSENBLUM SILVERMAN SUTTON S F
INC /CA, and Dodge & Cox are investment firms located in San
Francisco.


- 这太酷了。我们可以看到生成的Cypher。如果我们回到上面，我们会看到它与我们要求的非常相似。
- 好的，尝试相同的查询，但换一个城市。

In [22]:
prettyCypherChain("What investment firms are in Menlo Park?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'Menlo Park'
RETURN mgr.managerName
Full Context:
[{'mgr.managerName': 'Bordeaux Wealth Advisors LLC'}, {'mgr.managerName': 'Opes Wealth Management LLC'}, {'mgr.managerName': 'Solstein Capital, LLC'}, {'mgr.managerName': 'Stamos Capital Partners, L.P.'}, {'mgr.managerName': 'TIEMANN INVESTMENT ADVISORS, LLC'}, {'mgr.managerName': 'SCGE MANAGEMENT, L.P.'}, {'mgr.managerName': 'Nelson Capital Management, LLC'}, {'mgr.managerName': 'Jasper Ridge Partners, L.P.'}, {'mgr.managerName': 'CROSSLINK CAPITAL INC'}]

> Finished chain.
The investment firms in Menlo Park include Jasper Ridge
Partners, L.P. and Crosslink Capital Inc.


- 我们可以看到WHERE子句已更改为查找Menlo Park的字符串文字，并且得到了正确的答案。

- 让我们尝试一些我们还没有教它怎么做的事情。我们教它如何处理投资公司，那么公司呢？
- 让我们找出在圣克拉拉的公司。

In [23]:
prettyCypherChain("What companies are in Santa Clara?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (comp:Company)-[:LOCATED_AT]->(compAddress:Address)
    WHERE compAddress.city = 'Santa Clara'
RETURN comp.companyName
Full Context:
[{'comp.companyName': 'PALO ALTO NETWORKS INC'}, {'comp.companyName': 'SEAGATE TECHNOLOGY'}, {'comp.companyName': 'ATLASSIAN CORP PLC'}]

> Finished chain.
The companies in Santa Clara are PALO ALTO NETWORKS INC,
SEAGATE TECHNOLOGY, and ATLASSIAN CORP PLC.


- 我们没有专门教导 LLM 如何解决这个问题，但通过这个示例和了解图谱的模式，LLM能够生成一个Cypher查询，从一个位于地址的公司中找到一个模式匹配，这个地址城市是圣克拉拉，正如我们所要求的那样。
- 然后我们得到了结果。

- 这是关于公司而不是投资公司的一个变体。让我们尝试另一个变体。
- 如果你还记得的话，我们不是找出位于特定城市的东西，而是想通过计算距离找出靠近某个城市的东西。LLM 能够想出如何编写吗？

In [24]:
prettyCypherChain("What investment firms are near Santa Clara?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
WHERE mgrAddress.city = 'Santa Clara'
RETURN mgr.managerName
Full Context:
[]

> Finished chain.
I don't know the answer.


- 我们教它的只是如何进行模式匹配，如何进行WHERE子句以及如何返回值。所以它需要更多的指导才能回答这样的问题。
- 我们可以通过改变提示并给它更多的示例来做到这一点。这里，它不知道可以进行距离查询。让我们将其添加到提示中。

# 四、扩展提示以教会大模型新的 Cypher 模式

- 我们添加一个新示例。

In [25]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# What investment firms are near Santa Clara?
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress

The question is:
{question}"""

In [ ]:
CYPHER_GENERATION_TEMPLATE_zh = """任务：生成查询图数据库的 Cypher 语句。
指令：仅使用 Schema 提供的关系类型和属性。请勿使用任何没有提供的其他关系类型或属性。
Schema：{schema} 
注意：请勿在回复中包含任何解释或道歉。除了构建 Cypher 声明之外，请勿回答任何可能提出其他任何问题的问题。
除生成的 Cypher 语句外，请勿包含任何文本。
示例：以下是针对特定问题生成的 Cypher 语句的一些示例：

# 旧金山有哪些投资公司？
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# 圣克拉拉附近有哪些投资公司？
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress

问题是：
{question}"""

- 一旦我们改变了这个Cypher生成模板，我们还需要更新所有基于这个模板构建的内容。
- 每当您对 Cypher 生成模板进行更改时，请重新运行此代码！

In [26]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

cypherChain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

- 更新后，我们会再次问这个问题，看看LLM生成的是什么。让我们滚动查看完整结果。

In [27]:
prettyCypherChain("What investment firms are near Santa Clara?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (address:Address)
    WHERE address.city = "Santa Clara"
MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, managerAddress.location) < 10000
RETURN mgr.managerName, mgr.managerAddress
Full Context:
[{'mgr.managerName': 'Mine & Arao Wealth Creation & Management, LLC.', 'mgr.managerAddress': '901 CAMPISI WAY, SUITE 140, CAMPBELL, CA, 95008'}]

> Finished chain.
Mine & Arao Wealth Creation & Management, LLC. is an
investment firm near Santa Clara, located at 901 CAMPISI
WAY, SUITE 140, CAMPBELL, CA, 95008.


- 让我们仔细看看是否正确。我们希望投资公司在圣克拉拉附近，所以匹配地址城市是圣克拉拉。很好。这是我们想要的地方。
- 然后是模式匹配，找到位于某个地址的经理，现在距离计算是从地址位置开始（即圣克拉拉），以及经理的地址位置。
- 经理需要在10,000米以内得到了正确的查询，并且看起来结果也正确。
- 我觉得这真的很酷。GPT-3.5 看过足够的Cypher，即使只有两个示例，也能很好地生成它。

# 五、扩展查询以从 Form 10K 块中检索信息

- 我们可以在这里提供一个更多的示例，从公司连接到我们开始的 SEC 文件。我们知道我们拥有的第一批数据来自项目一，项目一讲的是公司实际做什么。
- 让我们问一个关于这个的问题。我们在研究Palo Alto Networks。例如，我们可以说，让我们教LLM回答这个问题，Palo Alto Networks做什么？然后我们可以问其他公司的这个问题。

- 我们将提供给LLM的Cypher示例将使用全文搜索找到公司的名称。这里拼写完全正确，Palo Alto Networks。
- 然后我们将从那家公司进行匹配，我们已经将节点重命名为com，从提交了某个表格的公司开始，然后继续这个表格，通过一个部分到一个块。
- 现在这一部分很重要，这一部分的关系是我们用来标识一个部分的开始，作为链表的头节点。所以我们希望这个部分的 F10KItem 称为 item 1。
- 这将带我们到 item 1 部分块的第一个块，返回该块的文本，这将是我们提供给LLM以实际回答问题的内容。

In [28]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# What investment firms are near Santa Clara?
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress

# What does Palo Alto Networks do?
  CALL db.index.fulltext.queryNodes(
         "fullTextCompanyNames", 
         "Palo Alto Networks"
         ) YIELD node, score
  WITH node as com
  MATCH (com)-[:FILED]->(f:Form),
    (f)-[s:SECTION]->(c:Chunk)
  WHERE s.f10kItem = "item1"
RETURN c.text

The question is:
{question}"""

In [ ]:
CYPHER_GENERATION_TEMPLATE_zh = """任务：生成查询图数据库的 Cypher 语句。
指令：仅使用 Schema 提供的关系类型和属性。请勿使用任何没有提供的其他关系类型或属性。
Schema：{schema} 
注意：请勿在回复中包含任何解释或道歉。除了构建 Cypher 声明之外，请勿回答任何可能提出其他任何问题的问题。
除生成的 Cypher 语句外，请勿包含任何文本。
示例：以下是针对特定问题生成的 Cypher 语句的一些示例：

# 旧金山有哪些投资公司？
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# 圣克拉拉附近有哪些投资公司？
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress

# 帕洛阿尔托网络是做什么的？
  CALL db.index.fulltext.queryNodes(
         "fullTextCompanyNames", 
         "Palo Alto Networks"
         ) YIELD node, score
  WITH node as com
  MATCH (com)-[:FILED]->(f:Form),
    (f)-[s:SECTION]->(c:Chunk)
  WHERE s.f10kItem = "item1"
RETURN c.text

问题是：
{question}"""

- 保存它，重新创建链，然后我们会问一个关于Palo Alto Networks做什么的问题。

In [29]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

cypherChain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)


- 现在让我们问这个问题，看看我们是否得到一个看起来像上面的Cypher查询。

In [30]:
prettyCypherChain("What does Palo Alto Networks do?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
CALL db.index.fulltext.queryNodes(
     "fullTextCompanyNames", 
     "Palo Alto Networks"
     ) YIELD node, score
WITH node as com
MATCH (com)-[:FILED]->(f:Form),
  (f)-[s:SECTION]->(c:Chunk)
WHERE s.f10kItem = "item1"
RETURN c.text
Full Context:
[{'c.text': '>Item 1. Business\nGeneral\nPalo Alto Networks, Inc. is a global cybersecurity provider with a vision of a world where each day is safer and more secure than the one before. We were incorporated in 2005 and are headquartered in Santa Clara, California.\nWe empower enterprises, organizations, service providers, and government entities to protect themselves against today’s most sophisticated cyber threats. Our cybersecurity platforms and services help secure enterprise users, networks, clouds, and endpoints by delivering comprehensive cybersecurity backed by industry-leading artificial intelligence and automation. We are a leading provider of zero trust solutions, star

- 让我们看看。它正在进行全文搜索，进行模式匹配。
- 正如我们所希望的那样，它生成了最终答案。

在继续之前，请暂停视频并尝试使用不同的提示进行实验，以及给出示例的 Cypher 查询，并提出不同的问题。看看LLM能否生成适合这些问题的Cypher，如果不能，就给它再提供几个示例，查看笔记本，找到相关示例，添加到原始模板，更新所有链，然后再次运行问题，看看得到什么结果。

In [31]:
# 检查图谱schema
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties are the following: Chunk {textEmbedding:
LIST, f10kItem: STRING, chunkSeqId: INTEGER, text: STRING,
cik: STRING, cusip6: STRING, names: LIST, formId: STRING,
source: STRING, chunkId: STRING},Form {cusip6: STRING,
names: LIST, formId: STRING, source: STRING},Company
{location: POINT, cusip: STRING, names: LIST,
companyAddress: STRING, companyName: STRING, cusip6:
STRING},Manager {location: POINT, managerName: STRING,
managerCik: STRING, managerAddress: STRING},Address
{location: POINT, country: STRING, city: STRING, state:
STRING} Relationship properties are the following: SECTION
{f10kItem: STRING},OWNS_STOCK_IN {shares: INTEGER,
reportCalendarOrQuarter: STRING, value: FLOAT} The
relationships are the following: (:Chunk)-[:NEXT]-
>(:Chunk),(:Chunk)-[:PART_OF]->(:Form),(:Form)-[:SECTION]-
>(:Chunk),(:Company)-[:FILED]->(:Form),(:Company)-
[:LOCATED_AT]->(:Address),(:Manager)-[:LOCATED_AT]-
>(:Address),(:Manager)-[:OWNS_STOCK_IN]->(:Company)


In [32]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# What investment firms are near Santa Clara?
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress

# What does Palo Alto Networks do?
  CALL db.index.fulltext.queryNodes(
         "fullTextCompanyNames", 
         "Palo Alto Networks"
         ) YIELD node, score
  WITH node as com
  MATCH (com)-[:FILED]->(f:Form),
    (f)-[s:SECTION]->(c:Chunk)
  WHERE s.f10kItem = "item1"
RETURN c.text

The question is:
{question}"""

In [ ]:
CYPHER_GENERATION_TEMPLATE_zh = """任务：生成查询图数据库的 Cypher 语句。
指令：仅使用 Schema 提供的关系类型和属性。请勿使用任何没有提供的其他关系类型或属性。
Schema：{schema} 
注意：请勿在回复中包含任何解释或道歉。除了构建 Cypher 声明之外，请勿回答任何可能提出其他任何问题的问题。
除生成的 Cypher 语句外，请勿包含任何文本。
示例：以下是针对特定问题生成的 Cypher 语句的一些示例：

# 旧金山有哪些投资公司？
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# 圣克拉拉附近有哪些投资公司？
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress

# 帕洛阿尔托网络是做什么的？
  CALL db.index.fulltext.queryNodes(
         "fullTextCompanyNames", 
         "Palo Alto Networks"
         ) YIELD node, score
  WITH node as com
  MATCH (com)-[:FILED]->(f:Form),
    (f)-[s:SECTION]->(c:Chunk)
  WHERE s.f10kItem = "item1"
RETURN c.text

问题是：
{question}"""

In [33]:
# 更新提示并重置 QA 链
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

cypherChain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [34]:
prettyCypherChain("<<REPLACE WITH YOUR QUESTION>>")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (company:Company)-[:FILED]->(form:Form)
WHERE company.companyName = 'Palo Alto Networks'
RETURN form.formId, form.source
Full Context:
[]

> Finished chain.
I'm sorry, but I don't have the information to provide an
answer to that question.
